In [73]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [74]:
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim=4, hidden_dim=128, action_dim=2):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, action_dim)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 輸出 logits (對應 action_dim=2)
        logits = self.fc3(x)
        return logits
    
    def get_action(self, state):
        """
        給定單一狀態 (shape: (4,))，
        輸出一個 action 與 log_prob。
        """
        # 轉成 batch=1 的張量
        state = torch.FloatTensor(state).unsqueeze(0)  # shape (1, 4)
        logits = self.forward(state)  # shape (1, 2)

        # 用 Categorical 分佈做采樣
        dist = torch.distributions.Categorical(logits=logits)
        action = dist.sample()          # 得到一個整數 0 or 1
        log_prob = dist.log_prob(action)
        
        return action.item(), log_prob


In [75]:
def discount_rewards(rewards, gamma=0.99):
    """
    給定一串 step 的 reward，例如 [r0, r1, r2, ...]，
    回傳每個 time step t 對應的折扣後回報 G_t。
    """
    discounted = np.zeros_like(rewards, dtype=np.float32)
    running_add = 0
    for t in reversed(range(len(rewards))):
        running_add = rewards[t] + gamma * running_add
        discounted[t] = running_add
    return discounted

In [76]:
def run_episode(env, policy_net, gamma=0.99):
    """
    跑一個 episode，收集 (log_prob, reward)。
    回傳：
      - log_probs: list of log_prob (tensor)
      - rewards: list of float
      - total_reward: episode 最後得到的累積 reward (評估用)
    """
    log_probs = []
    rewards = []
    total_reward = 0
    
    state = env.reset()[0]  # 若是 gymnasium，env.reset() 回傳 (obs, info)
    done = False
    
    while not done:
        action, log_prob = policy_net.get_action(state)
        
        # 執行動作
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        # 紀錄
        log_probs.append(log_prob)
        rewards.append(reward)
        
        total_reward += reward
        
        state = next_state
    
    # 計算折扣後回報
    discounted_r = discount_rewards(rewards, gamma)  # shape (episode_length,)
    
    return log_probs, discounted_r, total_reward


In [77]:
def train_cartpole(
    max_episodes=1000, 
    gamma=0.99, 
    lr=1e-3, 
    hidden_dim=128
):
    env = gym.make("CartPole-v1", render_mode="rgb_array")
    
    # 環境狀態維度=4，動作維度=2
    policy_net = PolicyNetwork(state_dim=4, hidden_dim=hidden_dim, action_dim=2)
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)
    
    for episode in range(max_episodes):
        log_probs, discounted_r, total_reward = run_episode(env, policy_net, gamma)
        
        # 準備計算 policy gradient loss
        # Σ_t [ -log_pi(a_t|s_t) * G_t ]
        # G_t 就是 discounted_r[t]
        loss = 0
        for log_prob, Gt in zip(log_probs, discounted_r):
            loss += -log_prob * Gt

        # 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 顯示訓練進度
        print(f"Episode {episode}, Reward = {total_reward}")
        
        # 如果總分連續多次都達到滿分(500)，可以提早結束
        if total_reward >= 500: 
            print("Solved CartPole!")
            break
    
    env.close()
    return policy_net


In [78]:
def test_policy(policy_net, episodes=5):
    env = gym.make("CartPole-v1")
    total_rewards = []
    
    for _ in range(episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        
        while not done:
            action, _ = policy_net.get_action(state)
            state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
        
        total_rewards.append(episode_reward)
    
    env.close()
    avg_reward = np.mean(total_rewards)
    print(f"Test over {episodes} episodes, average reward: {avg_reward:.2f}")
    return avg_reward


In [79]:
# 開始訓練
trained_policy = train_cartpole(max_episodes=500)

/opt/anaconda3/envs/rlenv/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0, Reward = 16.0
Episode 1, Reward = 14.0
Episode 2, Reward = 19.0
Episode 3, Reward = 15.0
Episode 4, Reward = 16.0
Episode 5, Reward = 25.0
Episode 6, Reward = 33.0
Episode 7, Reward = 16.0
Episode 8, Reward = 27.0
Episode 9, Reward = 17.0
Episode 10, Reward = 11.0
Episode 11, Reward = 11.0
Episode 12, Reward = 19.0
Episode 13, Reward = 22.0
Episode 14, Reward = 45.0
Episode 15, Reward = 12.0
Episode 16, Reward = 15.0
Episode 17, Reward = 31.0
Episode 18, Reward = 13.0
Episode 19, Reward = 22.0
Episode 20, Reward = 15.0
Episode 21, Reward = 15.0
Episode 22, Reward = 26.0
Episode 23, Reward = 44.0
Episode 24, Reward = 31.0
Episode 25, Reward = 22.0
Episode 26, Reward = 11.0
Episode 27, Reward = 26.0
Episode 28, Reward = 58.0
Episode 29, Reward = 28.0
Episode 30, Reward = 21.0
Episode 31, Reward = 18.0
Episode 32, Reward = 26.0
Episode 33, Reward = 28.0
Episode 34, Reward = 54.0
Episode 35, Reward = 12.0
Episode 36, Reward = 16.0
Episode 37, Reward = 24.0
Episode 38, Reward = 3

In [80]:
test_policy(trained_policy, episodes=10)

Test over 10 episodes, average reward: 451.50


451.5

In [81]:
def play_cartpole(policy_net):
    env = gym.make("CartPole-v1")
    state = env.reset()[0]
    done = False
    total_reward = 0
    
    while not done:
        env.render()
        action, _ = policy_net.get_action(state)
        state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward
    
    env.close()
    print(f"Total reward: {total_reward}")

play_cartpole(trained_policy)

Total reward: 132.0


/opt/anaconda3/envs/rlenv/lib/python3.12/site-packages/gym/envs/classic_control/cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
